| Trial | Steps | Interaction | Predicted Behavior | Memory Result (Description) |
| -------- | ------- | --- | --- | --- |
| A | Initial Fact | Tell the bot: "The passcode to the vault is 1234" | Stored in history	| Success |
| B | Distraction | First ask: "What is the capital of France?" Then ask: "Tell me a joke" | Pushes code out of window | Active |
| C | Recall Test | Ask: "What is the code to the vault?" | Should fail to recall | Forgotten |

Analysis:

  1. Compare Trial A to Trial C. Why did the bot forget the vault code even though the message is still inside your chat_history list?

        The bot forgot the vault code due to the the shifting of its context window, which is only able to capture 2 messages at a time, so as a result it only is able to "remember" the "Tell me a joke" and "What is the capital of France?" messages, not the passcode message.

2. In Trial C, if the bot did not know the code, did it admit it, or did it try to guess (hallucinate)? Explain what happens when a chatbot has a window that is too small.

    When the bot did not know the code, it admitted that it did not know the code. When a chatbot has a window that is too small, it will forget recent chats that it would otherwise remember if the window was larger.

3. Explain the role of [system_msg] in your code. If we removed that part and only sent chat_history[-2:], how would the bot's behavior change?

    The role of [system_msg] in the code (in my case [chat_history[0]) for the window is to make sure that the chatbot includes the message built into the bot for its role, for example in the Chatbot_Short_Term_Memory notebook the system message is that the bot is a science comedian, so it incorporates that message into all of its outputs. If we removed that part and only sent chat_history[-2:], then it would not take into account the system message and only include the last 2 repsonses in its thinking for the next prompt.

4. What if we want a smarter bot that remembers more? Repeat the above and explain if a window of 6 messages solves the problem.

    A window of 6 messages does solve the problem, as by the fourth input "What is the code to the vault?", it is able to call back to the original input that had the code within, getting it right. This is because a larger context window means it will be able to recall details from prompts further back in the conversation.


In [2]:
!pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.8 MB/s eta 0:00:00


In [4]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# 1. Setup
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    temperature=0.5
)

# 2. Initialize the History
# We start with the SystemMessage to set the persona
chat_history = [
    SystemMessage(content=" ")
]

print("--- (Type 'exit' to stop) ---")

# 3. The Chat Loop
while True:
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    # Add user's message to history
    chat_history.append(HumanMessage(content=user_input))

    # Get the AI response
    # We use .text here as a shortcut for Gemini 3 to get clean text
    response = llm.invoke(chat_history)
    ai_text = response.content[0]['text']

    print(f"Bot: {ai_text}")

    # Add the AI's response to history
    chat_history.append(AIMessage(content=ai_text))
    response = llm.invoke([chat_history[0]] + chat_history[-6:])


--- (Type 'exit' to stop) ---
You: The passcode to the vault is 1234


ChatGoogleGenerativeAIError: Error calling model 'gemini-3-flash-preview' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-3-flash\nPlease retry in 16.82982505s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-Fr